### Connect to google drive


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install missingno
!pip install geopy

### read 2019 Berline Airbnb Dataset
This file already contains only 2019 data , data reduced due to its extream size


In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings

# Ignore warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 500)

df = pd.read_csv('/content/drive/MyDrive/kaggle/berline/Airbnb Berlin.csv')
pd.set_option('display.max_columns', None)



### Data Prepeartion

### feature selection  (round #1)
in this section we going to drop all the.
* redundant / duplicate data.
* drop all the column that contains same value , these data is not useful.
* drop columns that almost not containing any data.
* drop all the data the clearly will not help me during this jurney , like (Urls , reviewer  id , reviewer name ... )

In [4]:
df = df.drop(columns=['Review ID', 'Reviewer ID', 'Reviewer Name', 'Listing URL','Listing Name','Host ID', 'Host URL', 'Host Name',
                           'City', 'Country Code', 'Country','First Review', 'Last Review', 'Square Feet', 'Business Travel Ready',
            ])

In [11]:
df['review_date'] = pd.to_datetime(df['review_date'])
df['year'] = df['review_date'].dt.year
df['month'] = df['review_date'].dt.month
df['day'] = df['review_date'].dt.day


In [12]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456961 entries, 0 to 456960
Data columns (total 35 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   index                 456961 non-null  int64         
 1   review_date           452805 non-null  datetime64[ns]
 2   Comments              452595 non-null  object        
 3   Listing ID            456961 non-null  int64         
 4   Host Since            456913 non-null  object        
 5   Host Response Time    398194 non-null  object        
 6   Host Response Rate    398194 non-null  float64       
 7   Is Superhost          456913 non-null  object        
 8   neighbourhood         456961 non-null  object        
 9   Neighborhood Group    456961 non-null  object        
 10  Postal Code           456961 non-null  object        
 11  Latitude              456961 non-null  float64       
 12  Longitude             456961 non-null  float64       
 13 

#Replace and clean values from numbers , like $,% and more

In [13]:
# remove the $ from price
df['Price'] = df['Price'].replace('[\$,]', '', regex=True).astype(float)

# remove the % from 'Host Response Rate'
df['Host Response Rate'] = df['Host Response Rate'].replace('[\%,]', '', regex=True).astype(float)

# Fix Postal Code incorrect values, remove '\n'and other irrelevant text
df['Postal Code'] = df['Postal Code'].astype(str).str[:5]

In [14]:
df.shape

(456961, 35)

In [15]:
# Define columns by type
rating_columns = [
      "Overall Rating","Accuracy Rating", "Cleanliness Rating", "Checkin Rating",
    "Communication Rating", "Location Rating", "Value Rating"
]

sum_columns = ["Reviews"]

categorical_columns = [
    "Host Response Time", "Neighborhood Group", "Room Type"]

integer_columns = ["Bedrooms","Beds","Bathrooms","Host Response Rate","Is Superhost","Latitude","Longitude",
                         "Accomodates", "Price", "Guests Included","Min Nights","Instant Bookable",
                         "Accomodates", "year","month","day","Host Since","Is Exact Location","Postal Code"
                         ,"Latitude","Latitude"]

# Aggregation functions
aggregation_functions = {
    **{col: "mean" for col in rating_columns},  # Mean for ratings
    **{col: "sum" for col in sum_columns},      # Sum for numerical values
    **{col: 'last' for col in integer_columns},  # Concat unique values
    **{col: lambda x: ", ".join(x.dropna().unique()) for col in categorical_columns}  # Concat unique values
}

# Group by 'Listing ID' and apply aggregation
df_aggregated = df.groupby("Listing ID").agg(aggregation_functions).reset_index()

df_aggregated.head(10)

,Listing ID,Overall Rating,Accuracy Rating,Cleanliness Rating,Checkin Rating,Communication Rating,Location Rating,Value Rating,Reviews,Bedrooms,Beds,Bathrooms,Host Response Rate,Is Superhost,Latitude,Longitude,Accomodates,Price,Guests Included,Min Nights,Instant Bookable,year,month,day,Host Since,Is Exact Location,Postal Code,Host Response Time,Neighborhood Group,Room Type
0,2695,100.0,10.0,10.0,10.0,10.0,9.0,10.0,49,1.0,1.0,1.0,50.0,f,52.54851,13.40455,2,17.0,1,2,f,2019.0,4.0,21.0,09-16-08,t,10437,within a day,Pankow,Private room
1,3176,92.0,9.0,9.0,9.0,9.0,10.0,9.0,20736,1.0,2.0,1.0,50.0,f,52.53500,13.41758,4,90.0,2,62,f,2018.0,10.0,29.0,10-19-08,t,10405,within a day,Pankow,Entire home/apt
2,7071,96.0,10.0,10.0,10.0,10.0,10.0,10.0,52441,1.0,2.0,1.0,100.0,t,52.54316,13.41509,2,33.0,1,2,f,2019.0,5.0,8.0,05-16-09,t,10437,within an hour,Pankow,Private room
3,9991,100.0,10.0,10.0,10.0,10.0,10.0,10.0,36,4.0,7.0,2.5,100.0,f,52.53303,13.41605,7,180.0,5,6,f,2018.0,7.0,23.0,08-25-09,f,10405,within a day,Pankow,Entire home/apt
4,14325,93.0,10.0,10.0,9.0,10.0,9.0,9.0,529,0.0,1.0,1.0,100.0,f,52.54785,13.40556,1,70.0,1,90,f,2018.0,11.0,1.0,11-18-09,t,10437,within a day,Pankow,Entire home/apt
5,16401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,2.0,1.0,NaN,f,52.51051,13.45785,2,120.0,1,30,f,NaN,NaN,NaN,12-03-09,t,10245,,Friedrichshain-Kreuzberg,Private room
6,16644,94.0,10.0,10.0,10.0,10.0,9.0,9.0,2304,1.0,3.0,1.0,NaN,f,52.50479,13.43510,5,90.0,2,60,f,2017.0,12.0,14.0,12-20-09,t,10997,,Friedrichshain-Kreuzberg,Entire home/apt
7,17904,96.0,10.0,10.0,10.0,10.0,10.0,9.0,56169,NaN,1.0,1.0,100.0,f,52.49548,13.42182,2,49.0,1,5,f,2019.0,4.0,17.0,01-08-10,f,12047,within an hour,NeukÃ¶lln,Entire home/apt
8,20858,88.0,9.0,9.0,9.0,9.0,10.0,9.0,3844,2.0,2.0,1.0,100.0,f,52.53695,13.40762,4,129.0,2,3,f,2019.0,4.0,15.0,01-18-10,t,10435,within an hour,Pankow,Entire home/apt
9,21869,95.0,9.0,10.0,10.0,9.0,9.0,9.0,3600,1.0,2.0,1.0,NaN,f,52.50273,13.43462,4,70.0,2,60,f,2016.0,6.0,5.0,12-20-09,t,10997,,Friedrichshain-Kreuzberg,Entire home/apt


### reduce the following wide catigories :
* Host Response Rate Grouped
* Overall Rating
* Neighbourhood Grouped
* property_types
* Postal Code

In [16]:
# Define bins and labels for 'Host Response Rate'
bins = [0, 50, 80, 95, 100]
labels = ["Low Response", "Moderate Response", "High Response", "Very High Response"]

# Apply pd.cut() to create a new binned column
df_aggregated["Host Response Rate Grouped Cleansed"] = pd.cut(df_aggregated["Host Response Rate"], bins=bins, labels=labels, include_lowest=True)

#  Reduce Categories
df_aggregated['Overall Rating Grouped Cleansed'] = df_aggregated['Overall Rating'].apply(lambda x: np.ceil(x/10))


# 2. Grouping neighbourhoods into Neighborhood Groups
# neighbourhood_mapping = df_aggregated.groupby('neighbourhood')['Neighborhood Group'].first()
# df_aggregated['Neighbourhood Grouped Cleansed'] = df_aggregated['neighbourhood'].map(neighbourhood_mapping)

# 3. Reducing Property Types
property_mapping = {
    "Villa": "Vacation Rental",
    "Cottage": "Vacation Rental",
    "Bungalow": "Vacation Rental",
    "Cabin": "Vacation Rental",
    "Tiny house": "Vacation Rental",
    "Earth house": "Vacation Rental",
    "Treehouse": "Vacation Rental",
    "Hut": "Vacation Rental",
    "Barn": "Vacation Rental",
    "Houseboat": "Boats & Houseboats",
    "Boat": "Boats & Houseboats",
    "Camper/RV": "Mobile/Alternative Lodging",
    "Cave": "Mobile/Alternative Lodging",
    "Pension (South Korea)": "Mobile/Alternative Lodging",
    "Casa particular (Cuba)": "Mobile/Alternative Lodging",
}

# Apply mapping and assign 'Other' to rare categories
top_property_types = [
    "Apartment", "Loft", "House", "Townhouse", "Condominium", "Serviced apartment",
    "Hotel", "Hostel", "Guesthouse", "Bed and breakfast", "Boutique hotel"
]

# df_aggregated['Property Type Cleansed'] = df_aggregated['Property Type'].apply(
#     lambda x: property_mapping.get(x, x) if x in top_property_types or x in property_mapping else "Other"
#)

# 4. Binning Postal Codes (first two digits represent broad area)
# df_aggregated['Postal Code Cleansed'] = df_aggregated['Postal Code'].astype(str).str[:2]  # Use only first 2 digits

### Transform data:

In [18]:
df_aggregated.shape

(23536, 32)

In [19]:
# Extracting years from date columns
df_aggregated['Host Since'] = pd.to_datetime(df_aggregated['Host Since'])
df_aggregated['Host Since Year Cleansed'] = df_aggregated['Host Since'].dt.year

#transform true/false into bool
df_aggregated['Instant Bookable Cleansed'] = df_aggregated['Instant Bookable'].replace({'t': True, 'f': False})
df_aggregated['Is Superhost Cleansed'] = df_aggregated['Is Superhost'].replace({'t': True, 'f': False})
df_aggregated['Is Exact Location Cleansed'] = df_aggregated['Is Exact Location'].replace({'t': True, 'f': False})
df_aggregated['Instant Bookable Cleansed'] = df_aggregated['Instant Bookable'].replace({'t': True, 'f': False})

In [20]:
df_aggregated.drop(columns=['Host Since', 'Postal Code',
                 'Is Exact Location'
                 ], inplace=True)

In [21]:
df_aggregated.shape

(23536, 33)

### Generating new features from existing data:
Calculate the distance of each listing from Berlin's center and join each listing to a group.
Each group will contain listings that are within a specific distance from the center

In [22]:
from geopy.distance import great_circle
def distance_to_mid(lat, lon):
    berlin_centre = (52.5027778, 13.404166666666667)
    accommodation = (lat, lon)
    return great_circle(berlin_centre, accommodation).km

df_aggregated['Distance From Center Cleansed'] = df_aggregated.apply(lambda row: round(distance_to_mid(row['Latitude'], row['Longitude']), 1), axis=1)
bins = [0, 0.5, 1, 2, 4, 8, 16]
labels = ["Center", "Center1", "Center2", "Center4", "Center8", "Center16"]

# Apply pd.cut() to create a new binned column
df_aggregated["Distance From Center Grouped Cleansed"] = pd.cut(df_aggregated["Distance From Center Cleansed"], bins=bins, labels=labels, include_lowest=True)
df_aggregated.drop(columns=["Distance From Center Cleansed"], inplace=True)


In [24]:
df_aggregated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23536 entries, 0 to 23535
Data columns (total 34 columns):
 #   Column                                 Non-Null Count  Dtype   
---  ------                                 --------------  -----   
 0   Listing ID                             23536 non-null  int64   
 1   Overall Rating                         18914 non-null  float64 
 2   Accuracy Rating                        18888 non-null  float64 
 3   Cleanliness Rating                     18892 non-null  float64 
 4   Checkin Rating                         18870 non-null  float64 
 5   Communication Rating                   18886 non-null  float64 
 6   Location Rating                        18871 non-null  float64 
 7   Value Rating                           18868 non-null  float64 
 8   Reviews                                23536 non-null  int64   
 9   Bedrooms                               23516 non-null  float64 
 10  Beds                                   23501 non-null  flo

### cleaning data (Round: #2)
dropping the following feature , after using them to calculate new clean features.

In [ ]:
# df.drop(columns=['Host Since', 'neighbourhood', 'Latitude', 'Longitude', 'Property Type', 'Postal Code',
#                  'Host Response Rate', 'Overall Rating','Instant Bookable', 'Is Superhost',
#                  'Is Exact Location','Distance From Center Cleansed',
#                  'Comments'], inplace=True)